<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> ✍ В этом модуле мы продолжим изучение библиотеки Pandas и рассмотрим более сложные методы работы с данными, которые помогают производить их анализ и подготовку. Модуль можно условно разделить на две части:

- в первой части мы поговорим о продвинутых методах Pandas, позволяющих производить более детальный анализ зависимостей в данных и делать на их основе выводы;
- вторая часть посвящена работе с несколькими таблицами и их объединению.</div>

Как мы уже говорили, этап подготовки данных играет важную роль в Data Science. Однако этот этап не пройдёт успешно без тщательного анализа зависимостей в данных. Для успешного преобразования таблицы вам необходимо иметь представление о том, как признаки связаны между собой и с целевым признаком, который вы хотели бы предсказать. Поэтому в первой части модуля мы рассмотрим <b>основные инструменты, позволяющие производить такой анализ, а также методы сортировки данных.</b>

Другая проблема заключается в том, что ваши данные не всегда могут быть представлены в виде одной таблицы. Такое нередко случается при работе с выгрузками из целой базы данных. Например, к вам в руки могут попасть несколько таблиц с информацией о клиентах, об их заказах и о поставщиках. Однако вашей целью может быть анализ данных о поставках в целом при использовании всей предоставленной информации. В таком случае <b>таблицы необходимо объединить в единую структуру</b> — об этом мы и поговорим во второй части модуля.

# Вспомним, с какими данными мы работаем

В этом модуле мы продолжим нашу работу с датасетом о продажах объектов недвижимости в Мельбурне и его пригородах. 
В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

- удалили столбцы index и Coordinates;
- создали признак средней площади одной комнаты MeanRoomsSquare и ввели коэффициент соотношения площади здания к площади участка — AreaRatio;
- преобразовали признак даты продажи Date в формат datetime и создали на его основе следующие столбцы: номер месяц продажи (MonthSale), номер дня недели - =- продажи (WeekdaySale), «признак-мигалку» выходного дня (Weekend);
- заменили признак года постройки здания YearBuilt на его возраст AgeBuilding;



- извлекли из признака адреса объекта Address новый признак подтипа улицы StreetType и удалили столбец с адресом;
- уменьшили число уникальных наименований агентств по недвижимости (SellerG), а также число пригородов (Suburb);
- выделили категориальные признаки и преобразовали их в тип данных сategory;
- заменили сокращённые названия категорий признака типа объекта Type на их полные названия (h — house, t —  townhouse, u — unit).

In [9]:
import pandas as pd

melb_df = pd.read_csv("data/melb_data_fe.csv")
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


Файл подгрузился верно, однако есть один нюанс. Давайте посмотрим на информацию о столбцах с помощью метода info():

In [11]:
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

None

Обратите внимание, что наше преобразование столбцов к типам datetime и category «слетело».

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Ответ на самом деле очень прост: csv-файл не хранит в себе информацию о типах данных столбцов, поэтому при чтении Pandas автоматически определяет тип данных столбца. Не забывайте об этом, обмениваясь преобразованными данными с вашими коллегами.</div>

А пока предлагаем вам самим совершить преобразование столбцов к нужным типам данных ↓

###  Задание 1.1

Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
4359
</code>
</details>

In [17]:
melb_df["Date"] = pd.to_datetime(melb_df["Date"])
melb_df["Date"].dt.quarter.value_counts()

Date
3    4873
2    4359
4    2329
1    2019
Name: count, dtype: int64

###   Задание 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.

В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
10
</code>
</details>

In [27]:
melb_df.nunique().sort_values()

Weekend               2
Type                  3
Method                5
WeekdaySale           7
Regionname            8
Rooms                 9
Bathroom              9
Car                  11
StreetType           11
MonthSale            12
Bedroom              12
CouncilArea          33
SellerG              50
Date                 58
Suburb              120
AgeBuilding         152
Postcode            198
Distance            202
Propertycount       311
BuildingArea        602
MeanRoomsSquare    1275
Landsize           1448
Price              2204
Lattitude          6503
AreaRatio          6521
Longtitude         7063
dtype: int64

In [29]:
cols_to_exclude = [
    "Date",
    "Rooms",
    "Bedroom",
    "Bathroom",
    "Car",
]  # список столбцов, которые мы не берём во внимание

In [30]:
max_unique = 150
for col in melb_df.columns:
    if col not in cols_to_exclude and melb_df[col].nunique() < 150:
        melb_df[col] = melb_df[col].astype("category")
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ